In [ ]:
import pandas as pd

train = pd.read_csv("train_klever.csv", sep=';')
data = pd.read_csv("data_klever.csv", sep=';')
test = pd.read_csv("test_klever.csv", sep=';')
res = train.merge(data, on="ID")
res.head()

,ID,Answer,Question
0,1,0,Как зовут лодочника на реке Стикс в древнегреч...
1,2,1,Как в химии обозначается свинец?
2,3,0,Какой химический элемент преобладает в составе...
3,4,0,Кто из перечисленных был пажом во времена Екат...
4,5,0,Когда началась 2 мировая война?


In [ ]:
res_test = test.merge(data, on="ID")
res_test.head()

,ID,Question
0,30001,"Что в литературе обозначает слово ""эпитет""?"
1,30002,Когда отмечается ДЕНЬ ШОКОЛАДА
2,30003,Кто из этих бодибилдеров играл Халка
3,30004,"Последняя на данный момент страна, официально ..."
4,30005,Как зовут лютоволка Арьи Старк из сериала «Иг...


# Функции для фичей!

In [ ]:
import nltk
import re
import string
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")
# проверка орфографии
!pip install pyspellchecker
import nltk
# nltk.download('punkt')
from spellchecker import SpellChecker
spell = SpellChecker(language='ru')
# !pip install pymorphy2
# import pymorphy2
nltk.download('punkt')

def remove_punct(text):
    # удаление пунктуации в тексте
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def txt_prep(df):
    """
    функция приводит весь текст к нижнему регистру
    удаляет пунктуацию
    удаляет стоп слова
    """

    df['Вопос без изменений'] = df['Question']
    df['Question'] = df['Question'].str.lower()
    df['Question'] = df['Question'].map(lambda x: remove_punct(x))
    df['Question'] = df['Question'].map(lambda x: x.split(' '))
    df['Question'] = df['Question'].map(lambda x: [token for token in x if token not in russian_stopwords\
                                                                    and token != " " \
                                                                    and token.strip() not in punctuation])
    df['Question'] = df['Question'].map(lambda x: ' '.join(x))
    df['Question'] = df['Question'].str.replace('»', '')
    df['Question'] = df['Question'].str.replace('«', '')
    return df

def feature_new(df):
    """
    генерируем признак оканчивается ли предложение вопросом или нет
    проверить начинаются ли предложения с заглавной буквы
    """
    l = []
    for index, row in df.iterrows():
        if row['Question'].strip().endswith("?"):
            flag = 1 # 1 - Yes
            l.append(flag)
        else:
            flag = 0 # 0 - No
            l.append(flag)

    df['Ending'] = l

    l = []
    for index, row in df.iterrows():
        if row['Question'][0].istitle():
            flag = 1 # 1 - Yes
            l.append(flag)
        else:
            flag = 0 # 0 - No
            l.append(flag)

    df['Begining'] = l
    return df


def orphog(res):
  """
  токенизация по словам и проверка орфографии
  чтобы орфография прафильно работала, надо убрать слова на англ
  также надо убрать ковычки "",которые в пунктцации не убрались
  """

  res['Word_mistake_cnt'] = 0
  lst1 = []
  lst2 = []

  for index,row in res.iterrows():
    words = nltk.word_tokenize(row["Question"])
    misspelled = spell.unknown(words)
    lst1.append(str(misspelled))
    lst2.append(len(misspelled))
  res['Word_mistake_cnt'] = lst2
  return None

def probel(X):
  """
  проверка наличия лишних пробелов(2 и более рядом)
  """
  probels = []
  for el in X.Question.str.split(" "):
    if "" in el:
      probels.append(1)
    else:
      probels.append(0)
  X["Probels"] = probels
  return None

def prob_punct(X):
    """
    проверка на наличие пробелов перед знаками препинания(плохой тон)
    """
    a = []
    for s in X["Question"]:
        pat = "\s+([{}]+)".format(re.escape(string.punctuation))
        res = re.sub("\s{2,}", " ", re.sub(pat, r"\1", s))
        if s == res:
            a.append(0)
        else:
            a.append(1)
    X["Prob_punct"] = a
    return None

def sign(df):
    """
    проверка на наличие восклицательных знаков(плохой тон)
    """
    a = []
    for s in df["Question"]:
        if len(re.findall(r"\!", s)) != 0:
            a.append(1)
        else:
            a.append(0)
    df["viskl_sign"] = a
    return None

def len_of_sent(X):
    """
    проверка длины предложения: хоть различия между средними, медианами
    не значимы, но нет вопросов от экспертов длиннее 20 слов и для длины
    предложения до 3 слов включительно, количество предложений от неэкспертов
    сильно больше в процентном соотношении
    """
    a = []
    for el in X.Question:
        if len(el.split(" ")) > 20 or len(el.split(" ")) < 4:
            a.append(0)
        else:
            a.append(1)
    X["Len_sent"] = a
    return None

def find_yo(X):
  """
  проверяет слова на наличие букв ё(гипотеза, что эксперты в словах, где нужна
  буква ё пишут именно ее а не е, а слов таких много)
  """
  a = []
  for el in X.Question:
    if "ё" in el:
      a.append(1)
    else:
      a.append(0)
  X["yo"] = a
  return None

def find_two_prep(X):
  """
  проверяет предложения на наличие двух и более подряд идущих знаков препинания)
  """

  a = []
  for el in X.Question:
    if re.search(r'["!#$%&()*+,-./:;<=>?@[\]^_`{|}~]{2}', el) is None:
      a.append(1)
    else:
      a.append(0)
  X["Two_prep"] = a
  return None

def quotes(X):
  """
  Проверяет кавычки(эксперты всегда пишут « »)
  """

  a = []
  for el in X.Question:
    if re.findall(r'\"', el):
      a.append(0)
    else:
      a.append(1)
  X["Quotes"] = a
  return None

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Дальше

In [ ]:
#добавляем фичи
res = feature_new(res)
orphog(res)
probel(res)
prob_punct(res)
sign(res)
len_of_sent(res)
find_yo(res)
quotes(res)
find_two_prep(res)
res

,ID,Answer,Question,Ending,Begining,Word_mistake_cnt,Probels,Prob_punct,viskl_sign,Len_sent,yo,Quotes,Two_prep
0,1,0,Как зовут лодочника на реке Стикс в древнегреч...,1,1,4,0,0,0,1,0,1,1
1,2,1,Как в химии обозначается свинец?,1,1,2,0,0,0,1,0,1,1
2,3,0,Какой химический элемент преобладает в составе...,1,1,3,0,0,0,1,0,1,1
3,4,0,Кто из перечисленных был пажом во времена Екат...,1,1,4,0,0,0,1,0,1,1
4,5,0,Когда началась 2 мировая война?,1,1,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,0,"Как называется игра, местом действия которой я...",1,1,1,0,0,0,1,0,1,1
29996,29997,0,Как в опере называют ведущую певицу?,1,1,2,0,0,0,1,0,1,1
29997,29998,0,На какой планете сутки длиннее года?,1,1,0,0,0,0,1,0,1,1
29998,29999,0,Сколько лет пролежал на печи известный богатыр...,1,1,3,0,0,0,1,0,1,1


In [ ]:
## чистим текст
res = txt_prep(res)
res

,ID,Answer,Question,Ending,Begining,Word_mistake_cnt,Probels,Prob_punct,viskl_sign,Len_sent,yo,Quotes,Two_prep,Вопос без изменений
0,1,0,зовут лодочника реке стикс древнегреческой миф...,1,1,4,0,0,0,1,0,1,1,Как зовут лодочника на реке Стикс в древнегреч...
1,2,1,химии обозначается свинец,1,1,2,0,0,0,1,0,1,1,Как в химии обозначается свинец?
2,3,0,химический элемент преобладает составе каменно...,1,1,3,0,0,0,1,0,1,1,Какой химический элемент преобладает в составе...
3,4,0,перечисленных пажом времена екатерины ii,1,1,4,0,0,0,1,0,1,1,Кто из перечисленных был пажом во времена Екат...
4,5,0,началась 2 мировая война,1,1,0,0,0,0,1,0,1,1,Когда началась 2 мировая война?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,0,называется игра местом действия которой являет...,1,1,1,0,0,0,1,0,1,1,"Как называется игра, местом действия которой я..."
29996,29997,0,опере называют ведущую певицу,1,1,2,0,0,0,1,0,1,1,Как в опере называют ведущую певицу?
29997,29998,0,планете сутки длиннее года,1,1,0,0,0,0,1,0,1,1,На какой планете сутки длиннее года?
29998,29999,0,сколько лет пролежал печи известный богатырь и...,1,1,3,0,0,0,1,0,1,1,Сколько лет пролежал на печи известный богатыр...


In [ ]:
#подготовка нужных столбцов

X_train = res.drop(['ID', 'Answer','Вопос без изменений'],axis=1)
y_train = res["Answer"]
X_test = res.drop(['ID', 'Answer','Вопос без изменений'],axis=1) # заглушка для обучения на всех данных
y_test = res["Answer"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split

#финальная модель обучалась на всем датасете, а не на сплите

#X_train, X_test, y_train, y_test = train_test_split(res.drop(['ID', 'Answer','Вопос без изменений'],axis=1),
                                                    #res['Answer'],
                                                    #test_size=0.3,
                                                    #random_state=42)

# возьмем tf idf
# Parameter election
ngram_range = (1,2)
max_features = 500

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        min_df=3,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

x_train_2 = tfidf.fit_transform(X_train['Question']).toarray()
x_test_2 = tfidf.transform(X_test['Question']).toarray()

# Добавляем еще признаки

x_train_2 = np.column_stack((x_train_2, np.array(X_train.Ending)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Begining)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Word_mistake_cnt)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Probels)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Prob_punct)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.viskl_sign)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Len_sent)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.yo)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Two_prep)))
x_train_2 = np.column_stack((x_train_2, np.array(X_train.Quotes)))

x_test_2 = np.column_stack((x_test_2, np.array(X_test.Ending)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Begining)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Word_mistake_cnt)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Probels)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Prob_punct)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.viskl_sign)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Len_sent)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.yo)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Two_prep)))
x_test_2 = np.column_stack((x_test_2, np.array(X_test.Quotes)))

In [ ]:
print(x_train_2.shape)
print(x_test_2.shape)

(21000, 510)
(9000, 510)


## CatBoost

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier
import sklearn

model_cb = CatBoostClassifier()
model_cb.fit(x_train_2, y_train)

#model_cb_preds = model_cb.predict_proba(x_test_2)[:,1]
#sklearn.metrics.roc_auc_score(y_test, model_cb_preds)

Learning rate set to 0.037803
0:	learn: 0.6614917	total: 49.2ms	remaining: 49.2s
1:	learn: 0.6143512	total: 93.4ms	remaining: 46.6s
2:	learn: 0.5891934	total: 137ms	remaining: 45.5s
3:	learn: 0.5663848	total: 186ms	remaining: 46.2s
4:	learn: 0.5319053	total: 230ms	remaining: 45.7s
5:	learn: 0.5123192	total: 274ms	remaining: 45.4s
6:	learn: 0.4958815	total: 319ms	remaining: 45.2s
7:	learn: 0.4809391	total: 365ms	remaining: 45.3s
8:	learn: 0.4658465	total: 418ms	remaining: 46.1s
9:	learn: 0.4534031	total: 463ms	remaining: 45.9s
10:	learn: 0.4412575	total: 515ms	remaining: 46.3s
11:	learn: 0.4310938	total: 559ms	remaining: 46.1s
12:	learn: 0.4217963	total: 614ms	remaining: 46.6s
13:	learn: 0.4133576	total: 664ms	remaining: 46.8s
14:	learn: 0.4055993	total: 709ms	remaining: 46.6s
15:	learn: 0.3984937	total: 753ms	remaining: 46.3s
16:	learn: 0.3838705	total: 796ms	remaining: 46s
17:	learn: 0.3781590	total: 840ms	remaining: 45.9s
18:	learn: 0.3731565	total: 888ms	remaining: 45.9s
19:	learn: 

0.7545146257298229

## XGB

In [ ]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=100)

xgb_clf.fit(x_train_2, y_train)

#xgb_clf_preds = xgb_clf.predict_proba(x_test_2)[:,1]

#sklearn.metrics.roc_auc_score(y_test, xgb_clf_preds)

0.760217650934876

# Тест

In [ ]:
res_test = feature_new(res_test)
orphog(res_test)
probel(res_test)
prob_punct(res_test)
sign(res_test)
len_of_sent(res_test)
find_yo(res_test)
find_two_prep(res_test)
quotes(res_test)
res_test = txt_prep(res_test)
res_test

,ID,Question,Ending,Begining,Word_mistake_cnt,Probels,Prob_punct,viskl_sign,Len_sent,yo,Two_prep,Quotes,Вопос без изменений
0,30001,литературе обозначает слово эпитет,1,1,4,0,1,0,1,0,0,0,"Что в литературе обозначает слово ""эпитет""?"
1,30002,отмечается день шоколада,0,1,1,0,0,0,1,0,1,1,Когда отмечается ДЕНЬ ШОКОЛАДА
2,30003,этих бодибилдеров играл халка,0,1,2,0,0,0,1,0,1,1,Кто из этих бодибилдеров играл Халка
3,30004,последняя данный момент страна официально прои...,0,1,3,0,0,0,1,0,1,1,"Последняя на данный момент страна, официально ..."
4,30005,зовут лютоволка арьи старк сериала игра престолов,1,1,4,1,1,0,1,0,1,1,Как зовут лютоволка Арьи Старк из сериала «Иг...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11082,41083,каком году распался ссср,0,1,3,0,0,0,1,0,1,1,В каком году распался СССР
11083,41084,сколько калорий 100 гр арбуза,1,1,3,0,0,0,1,0,1,1,Сколько калорий в 100 гр арбуза?
11084,41085,сколько хвостов лиса который является демоном ...,1,1,3,0,0,0,1,0,1,1,"Сколько хвостов у лиса, который является демон..."
11085,41086,сколько магнитогорский металлург становился об...,0,1,4,0,0,0,1,0,1,1,Сколько раз магнитогорский металлург становилс...


In [ ]:
res_test_2 = tfidf.transform(res_test['Question']).toarray()

# Добавляем еще признаки
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Ending)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Begining)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Word_mistake_cnt)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Probels)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Prob_punct)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.viskl_sign)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Len_sent)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.yo)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Two_prep)))
res_test_2 = np.column_stack((res_test_2, np.array(res_test.Quotes)))

In [ ]:
y_test_pred = xgb_clf.predict_proba(res_test_2)[:,1]
y_test_pred = pd.DataFrame(y_test_pred, columns=['pred'])
y_test_pred
y_test_pred
submition_XGB = pd.concat([test['ID'], y_test_pred], axis=1)

submition_XGB.to_csv('submition_XGB1.csv', index=False, header=False)
files.download("submition_XGB1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>